In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from model import MLP
from dataset_ffcv import get_dataloaders

/home/siddhesh1793/anaconda3/envs/clip_classifier/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load model based on output dir
model_num = 68
output_dir = "outputs/2022-11-09/17-24-47"
model_ckpt_filepath = os.path.join(output_dir, f"model_{model_num}.ckpt")
config_filepath = os.path.join(output_dir, "wandb")

files = os.listdir(config_filepath)
folder = None
for file in files:
	if "run-" in file:
		folder = file
		break

config_filepath = os.path.join(config_filepath, folder, "files", "config.yaml")


In [5]:
import yaml
with open(config_filepath, "r") as f:
	config = yaml.safe_load(f)

In [6]:
import torch
model = MLP(**config['model']['value']).to('cuda')
model_dict = torch.load(model_ckpt_filepath)
model.load_state_dict(torch.load(model_ckpt_filepath)['model_state_dict'])
model.eval()

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=1000, bias=False)
  )
)

In [7]:
# Create ffcv dataset objects
# config['dataloader']['value']['num_workers'] = 8
_, _, test_dl = get_dataloaders(**config['dataset']['value'], **config['dataloader']['value'])

In [11]:
def process_batch(model, data, f):
	# Get top 5 classification per image
	output = model(data[2])

	topk_val, topk_idxs = torch.topk(output, k=5, dim=-1)
	# TODO - consider doing softmax and then topk

	# Iterate through the batch and write the predictions to submission file f
	# Keep in mind that predictions are 1 indexed, so range from 1 to 1000
	topk_idxs += 1

	for idx in range(topk_idxs.shape[0]):
		list = topk_idxs[idx].cpu().numpy().tolist()
		if len(list) > 5:
			print (list)

		max_val = max(list)
		min_val = min(list)

		if max_val > 1000:
			print (list)

		if min_val < 1:
			print (list)

		topk_string = ' '.join([str(class_idx) for class_idx in topk_idxs[idx].cpu().numpy().tolist()]  + [" \n"])
		f.write(topk_string)

In [12]:
with open('predictions.txt', 'a') as f:
	for data in test_dl:
		process_batch(model, data, f)

In [13]:
with open('predictions.txt', 'r') as f:
	print (len(list(f.readlines())))

100000
